Disclaimer: This code should only be used as a reference and the responsibility of using this code lies solely on the user/users who is/ are using/executing it.

This program scrapes Metacritics, IMDB and Rotten Tomatoes for movies genres. then save them to CSVs. The library used is BeautifulSoup.

In [1]:
# Imports
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import numpy as np

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

no_movies = 10 # You can change this number to change to number of the movies scarped. 
               # It can accept value from 0 to 100.


# 1. Metacritics

Url for Metacritics: https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc

In [2]:
# The url to be entered is given above
print("You get three tries to make the url right! The right url is given in the markdown cell above.")
i = 0
while i<3:
    url = input('Enter url for Metacritics- ')
    if url == "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc": 
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('td', class_ = 'clamp-summary-wrap')


        movie_names = [] # list of first no_movies movies names
        movie_links = [] # list of first no_movies movies names to scrape genres
        movies = container[:no_movies] #here we get the top no_movies movies we want

        # Scraping movie names
        for movie in movies:
            name = movie.find('h3').text
            movie_names.append(name)

        # Scraping individual movie links
        for movie in movies:
            tag = movie.find('a', class_ = 'title')
            link = tag.get('href', None)
            movie_links.append(link)

        movie_genre = {}
        final_raw_genres = []
        genres=[]

        # Scraping the genres of all the movies
        for link in movie_links:
            url = "https://www.metacritic.com/"+link
            page = requests.get(url, headers = headers)
            soup = BeautifulSoup(page.content, "html.parser")
            genre_container = soup.find_all('div', class_ = 'genres')
            genre_container = genre_container[:]
            raw_genres = [span.text for span in genre_container]

            for i in raw_genres:
                temp = i.split(',')
                temp1 = [k.replace("\n","") for k in temp]
                temp1[0] = temp1[0][9:]
                for k in range(len(temp1)):
                    temp1[k]=temp1[k].strip().lower()
            final_raw_genres.append(temp1)


        # Dictionary with movies name as keys and corresponding genres as the values
        dict_movie_metacritics = {}
        for movie, gen in zip(movie_names, final_raw_genres):
            dict_movie_metacritics[movie] = gen

        # Getting the movies with most number of genres
        largest = 0
        val = None
        for i in dict_movie_metacritics.keys():
            if largest<len(dict_movie_metacritics[i]):
                largest = len(dict_movie_metacritics[i])
                val = i

        # Dataframa of genres with movie names as columns names
        df = pd.DataFrame(columns = list(dict_movie_metacritics.keys()))
        df[str(val)] = pd.Series(np.array(dict_movie_metacritics[val]))

        for i in dict_movie_metacritics.keys():
            df[str(i)] = pd.Series(np.array(dict_movie_metacritics[i]))

        # Saving to CSV
        df.to_csv('metacritics_genres.csv',index=False)
        break
        
    if i<2:
        if i ==1:
            print("You have only one try left! try Again")
        else:
            print("You have ",2-i," tries left! try Again")
    else:
        print("You have exceeded your number of tries! Exiting! Goodbye!")
        break
    i+=1

You get three tries to make the url right! The right url is given in the markdown cell above.
Enter url for Metacritics- https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc


In [3]:
try:
    print(dict_movie_metacritics)
except:
    print("You didn't run the previous block of code correctly! Please run that code block correctly by providing the right url from the markdown cell above before running ths code block!")

{'Citizen Kane': ['drama', 'mystery'], 'The Godfather': ['drama', 'thriller', 'crime'], 'Rear Window': ['mystery', 'thriller'], 'Casablanca': ['drama', 'romance', 'war'], 'Boyhood': ['drama'], 'Three Colors: Red': ['drama', 'mystery', 'romance'], 'Vertigo': ['mystery', 'thriller', 'romance'], 'Notorious': ['drama', 'thriller', 'romance', 'film-noir'], "Singin' in the Rain": ['comedy', 'romance', 'musical'], 'City Lights': ['drama', 'comedy', 'romance']}


# 2. IMDB

Url for IMDB: https://www.imdb.com/chart/top/?ref_=nv_mv_250

In [4]:
# The url to be entered is given above
print("You get three tries to make the url right! The right url is given in the markdown cell above.")
i = 0
while i<3:
    url = input('Enter url for IMDB- ')
    if url == "https://www.imdb.com/chart/top/?ref_=nv_mv_250":
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('td', class_ = 'titleColumn') 

        movie_names = [] # list of first no_movies movies names
        movie_links = [] # list of first no_movies movies names to scrape genres
        movies = container[:no_movies] #here we get the top 50 movies we want

        # Scraping movie names
        for movie in movies:
            name = movie.find('a').text
            movie_names.append(name)

        # Scraping individual movie links    
        for movie in movies:
            tag = movie.find('a')
            link = tag.get('href', None)
            movie_links.append(link)

        container_genre = {}
        final_raw_genres = {}
        genres_list=[]
        container = None

        # Scraping the genres of all the movies
        for link, movie in zip(movie_links, movie_names):
            url = "https://www.imdb.com/"+link
            page = requests.get(url, headers = headers)
            soup = BeautifulSoup(page.content, "html.parser")
            container = soup.find_all('div', class_= "see-more inline canwrap")
            final_raw_genres[movie] = container[1].find_all('a')

        # Dictionary with movies name as keys and corresponding genres as the values
        dict_movie_imdb = {}
        for i in final_raw_genres.keys():
            arr_genres = []
            for j in final_raw_genres[i]:
                arr_genres.append(j.text.strip().lower())
            dict_movie_imdb[i]=arr_genres


        # Getting the movies with most number of genres
        largest = 0
        val = None
        for i in dict_movie_imdb.keys():
            if largest<len(dict_movie_imdb[i]):
                largest = len(dict_movie_imdb[i])
                val = i
        # Dataframa of genres with movie names as columns names
        df = pd.DataFrame(columns = list(dict_movie_imdb.keys()))
        df[str(val)] = pd.Series(np.array(dict_movie_imdb[val]))

        for i in dict_movie_imdb.keys():
            df[str(i)] = pd.Series(np.array(dict_movie_imdb[i]))

        # Saving to CSV
        df.to_csv('imdb_genres.csv',index=False)
        break
        
    if i<2:
        if i ==1:
            print("You have only one try left! try Again")
        else:
            print("You have ",2-i," tries left! try Again")
    else:
        print("You have exceeded your number of tries! Exiting! Goodbye!")
        break
    i+=1

You get three tries to make the url right! The right url is given in the markdown cell above.
Enter url for IMDB- https://www.imdb.com/chart/top/?ref_=nv_mv_250


In [5]:
try:
    print(dict_movie_imdb)
except:
    print("You didn't run the previous block of code correctly! Please run that code block correctly by providing the right url from the markdown cell above before running ths code block!")

{'The Shawshank Redemption': ['drama'], 'The Godfather': ['crime', 'drama'], 'The Godfather: Part II': ['crime', 'drama'], 'The Dark Knight': ['action', 'crime', 'drama', 'thriller'], '12 Angry Men': ['crime', 'drama'], "Schindler's List": ['biography', 'drama', 'history'], 'The Lord of the Rings: The Return of the King': ['action', 'adventure', 'drama', 'fantasy'], 'Pulp Fiction': ['crime', 'drama'], 'The Good, the Bad and the Ugly': ['western'], 'The Lord of the Rings: The Fellowship of the Ring': ['action', 'adventure', 'drama', 'fantasy']}


# 3. Rotten Tomatoes



Url for Rotten Tomatoes: https://www.rottentomatoes.com/top/bestofrt/

In [6]:
# The url to be entered is given above
print("You get three tries to make the url right! The right url is given in the markdown cell above.")
i = 0
while i<3:
    url = input('Enter the url for Rotten Tomatoes- ')
    if url == "https://www.rottentomatoes.com/top/bestofrt/": 
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('a', class_="unstyled articleLink")
        container = container[43:-2]

        movie_names = [] # list of first no_movies movies names
        movie_links = [] # list of first no_movies movies names to scrape genres
        movies = container[:no_movies] #here we get the top 50 movies we want

        # Scraping movie names
        for movie in movies:
            movie=str(movie)
            start = [x for x, v in enumerate(movie) if v == '>'][0]
            end = [x for x, v in enumerate(movie) if v == '<'][1]
            movie_names.append(movie[start+2:end].strip())

        # Scraping individual movie links
        for movie in movies:
            link = movie.get('href', None)
            movie_links.append(link)

        movie_genre = {}
        final_raw_genres = []
        genres=[]
        # Scraping the genres of all the movies
        for link in movie_links:
            url = "https://www.rottentomatoes.com"+link
            page = requests.get(url, headers = headers)
            soup = BeautifulSoup(page.content, "html.parser")

            genre_container = soup.find_all('div', class_="meta-value genre")
            raw_genres = [span.text for span in genre_container]

            temp_genre = []
            for i in raw_genres:
                temp= i.split(',')
            for i in temp:
                temp_genre.append(i.strip())
            final_raw_genres.append(temp_genre)

        # Dictionary with movies name as keys and corresponding genres as the values
        dict_movie_rotten_tomatoes = {}
        for movie, gen in zip(movie_names, final_raw_genres):
            dict_movie_rotten_tomatoes[movie] = gen

        # Getting the movies with most number of genres
        largest = 0
        val = None
        for i in dict_movie_rotten_tomatoes.keys():
            if largest<len(dict_movie_rotten_tomatoes[i]):
                largest = len(dict_movie_rotten_tomatoes[i])
                val = i

        # Dataframa of genres with movie names as columns names
        df = pd.DataFrame(columns = list(dict_movie_rotten_tomatoes.keys()))
        df[str(val)] = pd.Series(np.array(dict_movie_rotten_tomatoes[val]))

        for i in dict_movie_rotten_tomatoes.keys():
            df[str(i)] = pd.Series(np.array(dict_movie_rotten_tomatoes[i]))

        # Saving to CSV
        df.to_csv('rotten_tomatoes_genres.csv',index=False)
        break


    if i<2:
        if i ==1:
            print("You have only one try left! try Again")
        else:
            print("You have ",2-i," tries left! try Again")
    else:
        print("You have exceeded your number of tries! Exiting! Goodbye!")
        break
    i+=1

You get three tries to make the url right! The right url is given in the markdown cell above.
Enter the url for Rotten Tomatoes- https://www.rottentomatoes.com/top/bestofrt/


In [7]:
try:
    print(dict_movie_rotten_tomatoes)
except:
    print("You didn't run the previous block of code correctly! Please run that code block correctly by providing the right url from the markdown cell above before running ths code block!")

{'Black Panther (2018)': ['fantasy', 'adventure', 'action'], 'Avengers: Endgame (2019)': ['fantasy', 'adventure', 'sci fi', 'action'], 'Us (2019)': ['horror', 'mystery and thriller'], 'Toy Story 4 (2019)': ['fantasy', 'kids and family', 'animation', 'adventure', 'comedy'], 'Lady Bird (2017)': ['drama', 'comedy'], 'Citizen Kane (1941)': ['drama'], 'Mission: Impossible - Fallout (2018)': ['adventure', 'mystery and thriller', 'action'], 'The Wizard of Oz (1939)': ['fantasy', 'kids and family', 'musical'], 'BlacKkKlansman (2018)': ['drama', 'comedy', 'crime'], 'Get Out (2017)': ['horror', 'mystery and thriller', 'comedy']}
